In [12]:
def WriteHeader(f, TaskName):
    f.write("#!/bin/bash\n\n")
    f.write("#SBATCH --partition=single\n")
    f.write("#SBATCH --ntasks-per-node=40\n")
    f.write("#SBATCH --time=2:00:00\n")
    f.write(f"#SBATCH --job-name={TaskName}\n")
    f.write("#SBATCH --error=%x.%j.err\n")
    f.write("#SBATCH --output=%x.%j.out\n")
    f.write("#SBATCH --mail-user=hzhao@teco.edu\n\n")

In [3]:
def write_jobs(f, job_counter, epsilon, ds, seed):
    line = f'python3 experiment.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --projectname FaultAnalysis &\n'
    f.write(line)
    job_counter += 1
    return job_counter

job_counter = 0
file_counter = 1

for epsilon in [0., 0.05, 0.1]:
    # Open the first file for the current epsilon
    f = open(f'run_baseline_pNN_{epsilon}_{file_counter}.sh', 'w')
    WriteHeader(f, f'pNN{epsilon}')

    for ds in range(13):
        for seed in range(10):
            if job_counter >= 40:
                # Close the current file and reset the job counter
                f.write("\nwait\n")
                f.close()
                file_counter += 1
                job_counter = 0
                
                # Open a new file
                f = open(f'run_baseline_pNN_{epsilon}_{file_counter}.sh', 'w')
                WriteHeader(f, f'pNN{epsilon}')
            
            job_counter = write_jobs(f, job_counter, epsilon, ds, seed)

    # If the loop exits but there are still jobs that haven't been written to a closed file
    if not f.closed:
        f.write("\nwait\n")
        f.close()
        job_counter = 0  # Reset the job counter for the next epsilon
        file_counter = 1  # Reset file counter for the next epsilon


In [13]:
def write_jobs(f, job_counter, etr, ete, ds, seed):
    line = f'python3 evaluation.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {etr} --e_test {ete} &\n'
    f.write(line)
    job_counter += 1
    return job_counter

job_counter = 0
file_counter = 1

f = open(f'run_evaluation_{file_counter:02d}.sh', 'w')
WriteHeader(f, f'FAEva')

for ds in range(13):
    for seed in range(10):
        for etr in [0., 0.05, 0.1]:
            for ete in [0., 0.05, 0.1]:
                
                
                if job_counter >= 40:
                    # Close the current file and reset the job counter
                    f.write("\nwait\n")
                    f.close()
                    file_counter += 1
                    job_counter = 0
                    
                    # Open a new file
                    f = open(f'run_evaluation_{file_counter:02d}.sh', 'w')
                    WriteHeader(f, f'FAEva')
                
                job_counter = write_jobs(f, job_counter, etr, ete, ds, seed)

# If the loop exits but there are still jobs that haven't been written to a closed file
if not f.closed:
    f.write("\nwait\n")
    f.close()

In [11]:
f = open(f'run_evaluation.sh', 'w')
for i in range(1,31):
    f.write(f'sbatch run_evaluation_{i:02d}.sh \n')
f.close()